In [23]:
import collections
import functools
import itertools
import math
import random
import operator

### Get a sieve of all numbers $n$ to indicate whether they are prime, with $0 \leq n \leq 2\times10^7$

In [3]:
def get_primes_eratosthenes_sieve(n):
    is_prime = [1] * (n + 1)
    p = 2
    is_prime[0] = 0
    is_prime[1] = 0
    while p * p <= n:
        if is_prime[p]:
            for i in range(p * 2, n + 1, p):
                is_prime[i] = 0
        p += 1
    prime_list = []
    for i in range(n + 1):
        if is_prime[i]:
            prime_list.append(i)
    return is_prime

is_prime = get_primes_eratosthenes_sieve(2 * 10 ** 7)

### Use Pollard's Rho algorithm for finding prime factorizations for any $n$

In [4]:
# https://github.com/zhangbo2008/python_algorithm2/blob/c53669703b957a079f100c12711f86f5fc2f9389/algorithms/factorization/pollard_rho.py
def pollard_rho_prime_factorization(x):
    def f(x):
        return x * x + 1

    def rho(n, x1=2, x2=2):
        if n % 2 == 0:
            return 2
        i = 0
        while True:
            x1 = f(x1) % n
            x2 = f(f(x2)) % n
            divisor = math.gcd(abs(x1 - x2), n)
            i += 1
            if divisor != 1:
                break
            if i > 500:
                x1 = random.randint(1, 10)
                x2 = random.randint(1, 10)
                i = 0
        return divisor

    def pollard_rho_rec(x, factors):
        if x == 1:
            return

        if is_prime[x]:
            factors.append(x)
            return

        divisor = rho(x, random.randint(1, 10), random.randint(1, 10))
        pollard_rho_rec(divisor, factors)
        pollard_rho_rec(x // divisor, factors)

    if x == 1 or x == 0:
        return {x: 1}
    factors = []
    pollard_rho_rec(x, factors)
    return collections.Counter(factors)

### Store prime factorizations for each $n$, with $2\leq n\leq 2\times10^7$ for rapid access 

In [6]:
N = 2 * 10 ** 7
prime_factorization = {n: pollard_rho_prime_factorization(n) for n in range(2, N + 1)}

### Define functions for the Extended Euclidean algorithm, the Chinese Remainder Theorem, and the lifting of roots modulo $p^k$ to $p^{k+1}$ by Hensel's Lemma 

In [96]:
# Compute modular inverse of a (mod b) via extended_euclidean_algorithm(a, b)[1] % b
def extended_euclidean_algorithm(a, b):
    if a == 0:
        return b, 0, 1
    else:
        gcd, x, y = extended_euclidean_algorithm(b % a, a)
        return gcd, y - (b // a) * x, x

# Pg. 22 of http://www.personal.psu.edu/rcv4/CENT.pdf
def chinese_remainder_theorem(linear_congruences):
    # Solves the system of linear congruences x = c_i (mod m_i)
    M = functools.reduce(operator.mul, [m_i for c_i, m_i in linear_congruences])
    result = 0
    for c_i, m_i in linear_congruences:
        M_i = M // m_i
        GCD, x, y = extended_euclidean_algorithm(m_i, M_i)
        result += c_i * y * M_i
    return result % M, M

# Hensel's Lemma (solutions to polynomial(x) ≡ 0 (mod p^k)): https://github.com/p4-team/crypto-commons/blob/master/crypto_commons/rsa/rsa_commons.py
def lift(f, df, p, k, previous):
    result = []
    for lower_solution in previous:
        dfr = df(lower_solution)
        fr = f(lower_solution)
        if dfr % p != 0:
            t = (-(extended_euclidean_algorithm(dfr, p)[1]) * (fr // p ** (k - 1))) % p
            result.append(lower_solution + t * p ** (k - 1))
        if dfr % p == 0:
            if fr % p ** k == 0:
                for t in range(0, p):
                    result.append(lower_solution + t * p ** (k - 1))
    return result

def hensel_lifting(f, df, p, k, base_solution: int | list = None):
    """
    Calculate solutions to f(x) = 0 (mod p^k) for prime p, where f is a polynomial function in x
    :param f: function
    :param df: derivative of function
    :param p: prime number
    :param k: power of prime
    :param base_solution: solution to return for k = 1 (assumed to be [1, p - 1] if not supplied)
    :return: possible solutions to f(x) = 0 mod p^k
    """
    if base_solution is None:
        solution = [1, p - 1]
    elif type(base_solution) is list:
        solution = base_solution
    else:
        solution = [base_solution]
    for i in range(2, k + 1):
        solution = lift(f, df, p, i, solution)
    return list(set(solution))

### For our use case of the Hensel's Lemma, $f = x^2-1$ and $\partial f_x=2x$ because we are solving $m\equiv m^{-1}\,(\text{mod }n)$

In [91]:
h, dh = lambda x: x * x - 1, lambda x: 2 * x

### Compute $I(n)$ for $n=\prod p_i^{\alpha_i}$ by first solving the congruences $\{x_i^2\equiv1\,(\text{mod }p_i^{\alpha_i}){\}$ individually using Hensel's Lemma, and then combining the resultant solutions for all $i$ via the Chinese Remainder Theorem to deduce possible values for $m$

In [94]:
def I(n):
    if is_prime[n]:
        return 1
    
    hensel_lemma_solutions = []
    for p in prime_factorization[n]:
        roots = hensel_lifting(h, dh, p, prime_factorization[n][p])
        if len(roots) == 0:
            return 1
        hensel_lemma_solutions.append((p, prime_factorization[n][p], roots))
    I_n_list = [chinese_remainder_theorem([(congruence_system[i], (lambda p, k: p ** k)(*list(prime_factorization[n].items())[i])) for i in range(len(congruence_system))]) for congruence_system in itertools.product(*[roots for p, k, roots in hensel_lemma_solutions])]
    if (n - 1, n) in I_n_list:
        I_n_list.remove((n - 1, n))
    return max([x for x, M in I_n_list]) if len(I_n_list) != 0 else 1

In [95]:
sum_I_n = 0
for n in range(3, N + 1):
    sum_I_n += I(n)
    
    # Display progress of function execution for large values of N
    if n % (2 * 10 ** 5) == 0:
        print('Progress:', n // (2 * 10 ** 5), '%', end='\r', flush=True)
        
sum_I_n

153651073760956